In [51]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from housecredit import HouseCredit
from scraping import DemandDeposit
import statistics


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

def get_prices(pdf,page, dictionary):
    doc = pdfplumber.open(pdf)
    if len(page)==1:
        prod_df=pd.DataFrame(doc.pages[page[0]].extract_table())
        def find_prices(df):
            for x in range(df.shape[0]):
                for y in range(df.shape[1]):
                    if 'Euros' in str(df[y][x]):
                        return y
        def find_index(df,dictionary):
            index_col=list()
            for x in range(df.shape[0]):
                for y in range(find_prices(df)):
                    if str(df[y][x])=='None' or str(df[y][x])=='' or 'Nota' in str(df[y][x]):
                        pass
                    else:
                        words = [word for word in str(df[y][x]).split(' ')]
                        for word in words:
                            for key in dictionary.keys():
                                for item in dictionary[key]:
                                    if word in item:
                                        index_col.append(y)
            return int(statistics.mode(index_col))
        def clean_df(df):
            for x in range(df.shape[0]):
                for y in df.columns:
                    if str(df[y][x])=='' or str(df[y][x])=='None':
                        df[y][x]=np.nan
            return df.dropna(axis='columns', how='all').dropna(axis='rows', how='all')
        prices_col=find_prices(prod_df)
        index_col=find_index(prod_df, house_credit_com)
        prices_df=pd.DataFrame()
        prices_df['Commissions']=prod_df[index_col]
        prices_df['Prices']=prod_df[prices_col]
        return clean_df(prices_df)
def get_pdf(link):
        remote = urlopen(Request(link)).read()
        memory = BytesIO(remote)
        return memory
def commission (output):
    prices_df = output
    commissions={}
    for i in range(len(prices_df)):
        if type(prices_df['Commissions'][i])==str:
            print(prices_df['Commissions'][i])
            if '\n-' in prices_df['Commissions'][i]:
                names=prices_df['Commissions'][i].split(sep='\n-')
                prices=['']+prices_df['Prices'][i].split(sep='\n')
                for name in names:
                    name=name.replace('\n','')
                for price in prices:
                    price=price.replace('/','')
                    price=price.replace('--','')
                return commissions.update({name: price for name, price in zip(names, prices)})
            elif '\n ' in prices_df['Commissions'][i]:
                names=prices_df['Commissions'][i].split(sep='\n ')
                prices=['']+prices_df['Prices'][i].split(sep='\n')
                for name in names:
                    name=name.replace('\n','')
                for price in prices:
                    price=price.replace('/','')
                    price=price.replace('--','')
                return commissions.update({name: price for name, price in zip(names, prices)})
            else:
                name = prices_df['Commissions'][i]
                price = prices_df['Prices'][i]
                name = name.replace('\n','')
                name = name.replace('\xa0','')
                price = str(price).replace('--','')
                price = str(price).replace('/','')
                return commissions.update({name: price})
                

In [76]:
def output(url, page, dictionary):
    doc=get_pdf(url)
    prices_df=get_prices(doc, page, dictionary)
    prices_df.reset_index(drop='index', inplace=True)
    return prices_df

In [77]:
prices_df=output('https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf',[6], house_credit_com )

In [74]:
commission(output)

2.1. Crédito à habitação e outros créditos hipotecários


In [78]:
commissions={}
for i in range(len(prices_df)):
    if type(prices_df['Commissions'][i])==str:
        if '\n-' in prices_df['Commissions'][i]:
            names=prices_df['Commissions'][i].split(sep='\n-')
            prices=['']+prices_df['Prices'][i].split(sep='\n')
            for name in names:
                name=name.replace('\n','')
            for price in prices:
                price=price.replace('/','')
                price=price.replace('--','')
            commissions.update({name: price for name, price in zip(names, prices)})
        elif '\n ' in prices_df['Commissions'][i]:
            names=prices_df['Commissions'][i].split(sep='\n ')
            prices=['']+prices_df['Prices'][i].split(sep='\n')
            for name in names:
                name=name.replace('\n','')
            for price in prices:
                price=price.replace('/','')
                price=price.replace('--','')
            commissions.update({name: price for name, price in zip(names, prices)})
        else:
            name = prices_df['Commissions'][i]
            price = prices_df['Prices'][i]
            name = name.replace('\n','')
            name = name.replace('\xa0','')
            price = price.replace('--','')
            price = price.replace('/','')
            commissions.update({name: price})

AttributeError: 'float' object has no attribute 'replace'